In [3]:
import pymysql
from sqlalchemy import create_engine
import xgboost as xgb
from numpy import argmax
from sklearn import preprocessing
from xgboost import XGBClassifier #sklearn中调用XGBoost的接口类，XGBClassifier就是对xgboost的封装，内核是同一个内核
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from matplotlib import pyplot
import seaborn as sns
import time
import datetime
import matplotlib.pyplot as plt
import importlib
import dateutil.relativedelta
from sklearn.model_selection import train_test_split
import sys
import copy
import math
import os
from xgboost import plot_importance
from sklearn.model_selection import KFold
from numpy import argmax
import gc
from sklearn.externals import joblib
import pickle
import json
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import fbeta_score, make_scorer
from scipy.special import logit

/home/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def merge_all(tab_list,how,on):
    for i in range(len(tab_list)):
        if (i ==0):
            final = tab_list[0]
        else:
            final = final.merge(tab_list[i],how =how,on = on)
    return final


In [5]:
def get_score(val):
    return round(500 - (20/math.log(2)) * (math.log(15)+logit(val)),0)

In [6]:
def round_6(pred_rst):
    pred_rst = pred_rst.tolist()
    for i,v in enumerate(pred_rst):
        pred_rst[i] = round(float(v),6)
    return pred_rst

# 加载当天可以打分的用户

In [7]:
import xh_xw_v3_get_uid as get_uid
importlib.reload(get_uid)

<module 'xh_xw_v3_get_uid' from '/home/erick.wang/xh_qb/ziyouhua/xh_xw_v3/xw_xh_v3_model_builder/xh_xw_v3_get_uid.py'>

In [8]:
dafen_uid = get_uid.get_uid_daily('2019-11-14')

repay date is 2019-11-14


/home/erick.wang/xh_qb/ziyouhua/xh_xw_v3/xw_xh_v3_model_builder/xh_xw_v3_get_uid.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  uid_select.sort_values(by=['uid','repay_date'],inplace=True)


process 19871 user
2019-11-19 15:59:49,372 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2019-11-19 15:59:49,373 INFO sqlalchemy.engine.base.Engine {}
2019-11-19 15:59:49,385 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2019-11-19 15:59:49,386 INFO sqlalchemy.engine.base.Engine {}
2019-11-19 15:59:49,396 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2019-11-19 15:59:49,398 INFO sqlalchemy.engine.base.Engine {}
2019-11-19 15:59:49,405 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2019-11-19 15:59:49,406 INFO sqlalchemy.engine.base.Engine {}
2019-11-19 15:59:49,412 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2019-11-19 15:59:49,413 INFO sqlalchemy.engine.base.Engine {}
2019-11-19 15:59:49,419 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS a

2019-11-19 16:03:13,918 INFO sqlalchemy.engine.base.Engine COMMIT
2019-11-19 16:03:13,923 INFO sqlalchemy.engine.base.Engine 
        CREATE TABLE `xh_xw_v3_driver_score` (
        `id` bigint(20) unsigned NOT NULL AUTO_INCREMENT,
        `uid` bigint(20) NOT NULL DEFAULT '0',
        `p0`  bigint(20) unsigned NOT NULL DEFAULT '0',
        `p1`  bigint(20) unsigned NOT NULL DEFAULT '0',
        `p2`  bigint(20) unsigned NOT NULL DEFAULT '0',
        `p3`  bigint(20) unsigned NOT NULL DEFAULT '0',
        `stat0` datetime NOT NULL DEFAULT '2000-01-01 00:00:00', 
        `stat1` datetime NOT NULL DEFAULT '2000-01-01 00:00:00', 
        `stat2` datetime NOT NULL DEFAULT '2000-01-01 00:00:00', 
        `stat3` datetime NOT NULL DEFAULT '2000-01-01 00:00:00', 
        `repay0` datetime NOT NULL DEFAULT '2000-01-01 00:00:00', 
        `repay1` datetime NOT NULL DEFAULT '2000-01-01 00:00:00', 
        `repay2` datetime NOT NULL DEFAULT '2000-01-01 00:00:00', 
        `repay3` datetime NOT NUL

In [9]:
dafen_uid.head()

,uid,p3,repay3,stat3,repay2,stat2,p2,repay1,stat1,p1,repay0,stat0,p0,balance_0,acount_status_4_5,overdue_30,overdue_curr,excl_code,risk_level
28877,26880658926223360,201911,2019-11-14,2019-11-10,2019-10-14,2019-10-10,201910,2019-09-14,2019-09-10,201909,2019-08-14,2019-08-10,201908,0,0,0,0,No,666
39056,26883085721419776,201911,2019-11-14,2019-11-10,2019-10-14,2019-10-10,201910,2019-09-14,2019-09-10,201909,2019-08-14,2019-08-10,201908,1,0,0,0,E12,-988
69036,26884245889761280,201911,2019-11-14,2019-11-10,2019-10-14,2019-10-10,201910,2019-09-14,2019-09-10,201909,2019-08-14,2019-08-10,201908,1,0,0,0,E12,-988
101811,26885252134486017,201911,2019-11-14,2019-11-10,2019-10-14,2019-10-10,201910,2019-09-14,2019-09-10,201909,2019-08-14,2019-08-10,201908,0,0,0,0,No,666
59473,26887834334937088,201911,2019-11-14,2019-11-10,2019-10-14,2019-10-10,201910,2019-09-14,2019-09-10,201909,2019-08-14,2019-08-10,201908,1,0,0,0,E12,-988


In [10]:
uid_have_score = dafen_uid[dafen_uid.risk_level == 666][['uid','p3','excl_code']]

In [11]:
uid_no_score = dafen_uid[dafen_uid.risk_level != 666][['uid','p3','excl_code','risk_level']]

# 获取三方/网络变量/还款行为/贷中交易

## 获取同盾

In [15]:
import xh_xw_v3_get_td as get_td
importlib.reload(get_td)

<module 'xh_xw_v3_get_td' from '/home/erick.wang/xh_qb/ziyouhua/xh_xw_v3/xw_xh_v3_model_builder/xh_xw_v3_get_td.py'>

In [16]:
all_td = get_td.get_td_data(uid_have_score)

In [18]:
all_td.head()

,uid,xy_user_id,dzdate,td_6mon_bank,td_3mon_other,td_7day_xd,td_6mon_bank_avg,td_3mon_other_avg,td_7day_xd_avg
0,26880658926223360,2.688066e+16,2019-11-16 02:49:04,6.0,0.0,0.0,5.000000,0.666667,0.666667
1,26885252134486017,2.688525e+16,2019-11-16 01:57:20,4.0,0.0,0.0,4.333333,0.000000,0.000000
2,27453088686948352,2.745309e+16,2019-11-16 03:09:10,8.0,0.0,0.0,8.000000,0.000000,0.666667
3,30294704807493632,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,43370097026809856,4.337010e+16,2019-11-16 01:38:50,4.0,0.0,0.0,3.666667,0.000000,0.333333


## 获取互金

In [19]:
import xh_xw_v3_get_nifa as get_nifa
importlib.reload(get_nifa)

<module 'xh_xw_v3_get_nifa' from '/home/erick.wang/xh_qb/ziyouhua/xh_xw_v3/xw_xh_v3_model_builder/xh_xw_v3_get_nifa.py'>

In [20]:
all_nifa = get_nifa.get_nifa_data(uid_have_score)

In [21]:
all_nifa.head()

,uid,query_tm_x,loancount,overduecount,overdueamt,outstandcount,loanbal,query_tm_y,pre_loan_cnt_6m,loancount_avg,overduecount_avg,overdueamt_avg,outstandcount_avg,loanbal_avg,pre_loan_cnt_6m_avg
0,26880658926223360,2019-11-16 02:49:01,6.0,0.0,0.0,6.0,54262.0,2019-11-16 02:49:01,22.0,4.333333,0.0,0.0,4.333333,52026.000000,19.000000
1,26885252134486017,2019-11-16 01:57:17,20.0,0.0,0.0,10.0,79743.0,2019-11-16 01:57:17,1.0,18.333333,0.0,0.0,9.000000,64881.333333,1.000000
2,27453088686948352,2019-11-16 04:31:03,4.0,0.0,0.0,2.0,1069.0,2019-11-16 04:31:03,8.0,4.000000,0.0,0.0,2.000000,2795.000000,7.666667
3,30294704807493632,2019-11-16 04:08:04,38.0,8.0,8929.0,13.0,68748.0,2019-11-16 04:08:04,6.0,38.000000,8.0,8929.0,13.000000,68748.000000,6.000000
4,43370097026809856,2019-11-16 01:38:47,3.0,0.0,0.0,2.0,12199.0,2019-11-16 01:38:47,2.0,2.333333,0.0,0.0,1.333333,6068.666667,4.666667


## 获取详单网络变量

In [24]:
import xh_xw_v3_get_net as get_net
importlib.reload(get_net)

<module 'xh_xw_v3_get_net' from '/home/erick.wang/xh_qb/ziyouhua/xh_xw_v3/xw_xh_v3_model_builder/xh_xw_v3_get_net.py'>

In [25]:
all_net = get_net.get_net_data(uid_have_score)

In [26]:
all_net.head()

,uid,info_date,n_law_org_2nd,rt_exp_org_2nd,rt_ecm_org_2nd,rt_yd_xd_2nd,rt_kd_zd_2nd,n_sales_1st,n_other_org_1st,rt_other_org_1st,...,indgr_rt_avg,rt_kd_xd_1st_avg,rt_midblktel_2nd_avg,rt_trv_org_2nd_avg,n_fin_org_2nd_avg,rt_agency_2nd_avg,rt_cblk_appl_1st_avg,n_agency_1st_avg,rt_ecm_org_1st_avg,n_telphone_1st_avg
0,26880658926223360,2019-11-15 01:21:27,0.0,12.0,0.0,2565.0,1787.0,25.0,5.0,323.0,...,6208.0,NaN,146.666667,0.000000,0.0,12.333333,0.000000,2.000000,0.0,24.0
1,26885252134486017,2019-11-15 01:18:05,0.0,0.0,0.0,2790.0,1896.0,31.0,3.0,161.0,...,7681.0,NaN,162.666667,0.000000,0.0,0.000000,NaN,2.000000,108.0,16.0
2,27453088686948352,2019-11-15 01:22:30,0.0,27.0,0.0,3000.0,1616.0,11.0,2.0,351.0,...,5764.0,NaN,131.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,2.0
3,30294704807493632,2019-11-15 01:24:51,1.0,7.0,5.0,2526.0,1648.0,338.0,12.0,73.0,...,5908.0,0.0,88.333333,6.000000,67.0,55.000000,5859.333333,17.333333,6.0,166.0
4,43370097026809856,2019-11-15 01:16:53,2.0,17.0,34.0,2367.0,1544.0,13.0,2.0,72.0,...,4502.0,0.0,60.000000,11.666667,19.0,89.000000,2857.000000,1.000000,0.0,50.0


## 获取还款行为

In [33]:
import xh_xw_v3_get_repay as get_repay
importlib.reload(get_repay)

<module 'xh_xw_v3_get_repay' from '/home/erick.wang/xh_qb/ziyouhua/xh_xw_v3/xw_xh_v3_model_builder/xh_xw_v3_get_repay.py'>

In [34]:
repay_data = get_repay.get_repay_data(uid_have_score)

rep_cnt_auto_fail: 2019-11-19 16:20:08
rep_cnt_auto: 2019-11-19 16:20:10
rep_cnt_manual_succ: 2019-11-19 16:20:12
time4_rep_cnt_auto_succ: 2019-11-19 16:20:14
time2_rep_cnt: 2019-11-19 16:20:17
time5_rep_cnt: 2019-11-19 16:20:19
rep_cnt_manual: 2019-11-19 16:20:22
time4_rep_cnt_fail: 2019-11-19 16:20:25
rep_cnt_succ: 2019-11-19 16:20:27
rep_cnt_auto_succ: 2019-11-19 16:20:30
time5_rep_cnt_succ: 2019-11-19 16:20:33
rep_cnt: 2019-11-19 16:20:35
rep_cnt_fail: 2019-11-19 16:20:38
time4_rep_cnt: 2019-11-19 16:20:41
time5_rep_cnt_auto_succ: 2019-11-19 16:20:44


In [35]:
repay_data.head()

,uid,rep_cnt_auto_fail_avg_3m,rep_cnt_auto_fail_tot_3m,rep_cnt_auto_fail_max_3m,rep_cnt_auto_fail_std_3m,rep_cnt_auto_fail_nci_3m,rep_cnt_auto_fail_ncd_3m,rep_cnt_auto_fail_cav_3m,rep_cnt_auto_fail_cmn_3m,rep_cnt_auto_fail_cmx_3m,...,time5_rep_cnt_auto_succ_tnd_3m,time5_rep_cnt_auto_succ_mai_3m,time5_rep_cnt_auto_succ_mad_3m,time5_rep_cnt_auto_succ_cva_3m,time5_rep_cnt_auto_succ_cmm_3m,time5_rep_cnt_auto_succ_cmp_3m,time5_rep_cnt_auto_succ_cnm_3m,time5_rep_cnt_auto_succ_cnp_3m,time5_rep_cnt_auto_succ_cxm_3m,time5_rep_cnt_auto_succ_cxp_3m
0,26880658926223360,0.00,0,0,0.00,0.0,0.0,-99.0,-99.0,-99.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,0,-99.0,0,-99.0
1,26885252134486017,0.00,0,0,0.00,0.0,0.0,-99.0,-99.0,-99.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,0,-99.0,0,-99.0
2,27453088686948352,0.00,0,0,0.00,0.0,0.0,-99.0,-99.0,-99.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,0,-99.0,0,-99.0
3,30294704807493632,1.67,5,3,1.15,1.0,0.0,1.8,3.0,1.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,0,-99.0,0,-99.0
4,43370097026809856,0.00,0,0,0.00,0.0,0.0,-99.0,-99.0,-99.0,...,0.0,0.0,0.0,-99.0,-99.0,-99.0,0,-99.0,0,-99.0


## 获取贷中交易

In [30]:
import xh_xw_v3_get_transaction as get_transaction
importlib.reload(get_transaction)

<module 'xh_xw_v3_get_transaction' from '/home/erick.wang/xh_qb/ziyouhua/xh_xw_v3/xw_xh_v3_model_builder/xh_xw_v3_get_transaction.py'>

In [31]:
transaction_data = get_transaction.get_transaction_data(uid_have_score)

trans_amt_per_day_max: 2019-11-19 16:15:47
trans_amt_per_cnt_holiday: 2019-11-19 16:15:50
trans_amt_per_cnt_20_500: 2019-11-19 16:15:53
cons_days: 2019-11-19 16:15:56
ca_amt_per_cnt: 2019-11-19 16:15:59
trans_amt_per_cnt_weekday: 2019-11-19 16:16:02
trans_cnt_20_500: 2019-11-19 16:16:06
trans_amt_refuse_wx: 2019-11-19 16:16:09
cons_amt_refuse: 2019-11-19 16:16:12
trans_hours_gap_per_day_avg: 2019-11-19 16:16:15
trans_cnt_refuse: 2019-11-19 16:16:19
trans_amt_per_cnt_1day_max: 2019-11-19 16:16:22
cons_amt: 2019-11-19 16:16:25
trans_cheat_rule_num: 2019-11-19 16:16:29
trans_amt_per_day_min: 2019-11-19 16:16:32
trans_amt_500_2k: 2019-11-19 16:16:36
trans_days: 2019-11-19 16:16:39
trans_amt_weekday: 2019-11-19 16:16:43
trans_amt_per_cnt_union: 2019-11-19 16:16:46
trans_amt_per_cnt_qr: 2019-11-19 16:16:50
trans_amt_per_cnt_min: 2019-11-19 16:16:54
trans_amt_wx: 2019-11-19 16:16:57
install_amt: 2019-11-19 16:17:01
cons_amt_per_cnt_refuse: 2019-11-19 16:17:04
ca_amt_per_day: 2019-11-19 16:17:

In [32]:
transaction_data.head()

,uid,trans_amt_per_day_max_avg_3m,trans_amt_per_day_max_tot_3m,trans_amt_per_day_max_max_3m,trans_amt_per_day_max_std_3m,trans_amt_per_day_max_nci_3m,trans_amt_per_day_max_ncd_3m,trans_amt_per_day_max_cav_3m,trans_amt_per_day_max_cmn_3m,trans_amt_per_day_max_cmx_3m,...,trans_amt_workday_worktime_mad_3m,trans_amt_workday_worktime_cva_3m,trans_amt_workday_worktime_cmm_3m,trans_amt_workday_worktime_cmp_3m,trans_amt_workday_worktime_cnm_3m,trans_amt_workday_worktime_cnp_3m,trans_amt_workday_worktime_cxm_3m,trans_amt_workday_worktime_cxp_3m,ratio_avg_3m,ratio_tnd_3m
0,26880658926223360,346666.67,1040000,800000,410528.12,1.0,1.0,0.69,-99.00,0.3,...,0.0,-99.00,-99.00,-99.00,0,-99.0,0,-99.0,0.93,-0.05
1,26885252134486017,300000.00,900000,300000,0.00,0.0,0.0,1.00,1.00,1.0,...,0.0,-99.00,-99.00,-99.00,0,-99.0,0,-99.0,0.94,0.08
2,27453088686948352,26833.33,80500,42000,23304.15,1.0,1.0,1.57,-99.00,1.0,...,0.0,-99.00,-99.00,-99.00,0,-99.0,0,-99.0,0.00,0.00
3,30294704807493632,0.00,0,0,0.00,0.0,0.0,-99.00,-99.00,-99.0,...,0.0,-99.00,-99.00,-99.00,0,-99.0,0,-99.0,NaN,NaN
4,43370097026809856,515960.00,1547880,1500000,852203.82,2.0,0.0,2.91,64.16,1.0,...,-7440.0,1.08,2.15,1.15,18800,-99.0,0,0.0,0.66,0.06


# 最终数据

In [36]:
dafen_data = merge_all([uid_have_score,all_td,all_nifa,all_net,transaction_data,repay_data],on='uid',how='left')

In [37]:
dafen_data.replace(-99,np.nan,inplace=True)
dafen_data.replace(-98,np.nan,inplace=True)
dafen_data.replace(-1,np.nan,inplace=True)

In [38]:
dafen_data.shape

(6681, 1754)

In [39]:
dafen_data.head()

,uid,p3,excl_code,xy_user_id,dzdate,td_6mon_bank,td_3mon_other,td_7day_xd,td_6mon_bank_avg,td_3mon_other_avg,...,time5_rep_cnt_auto_succ_tnd_3m,time5_rep_cnt_auto_succ_mai_3m,time5_rep_cnt_auto_succ_mad_3m,time5_rep_cnt_auto_succ_cva_3m,time5_rep_cnt_auto_succ_cmm_3m,time5_rep_cnt_auto_succ_cmp_3m,time5_rep_cnt_auto_succ_cnm_3m,time5_rep_cnt_auto_succ_cnp_3m,time5_rep_cnt_auto_succ_cxm_3m,time5_rep_cnt_auto_succ_cxp_3m
0,26880658926223360,201911,No,2.688066e+16,2019-11-16 02:49:04,6.0,0.0,0.0,5.000000,0.666667,...,0.0,0.0,0.0,NaN,NaN,NaN,0,NaN,0.0,NaN
1,26885252134486017,201911,No,2.688525e+16,2019-11-16 01:57:20,4.0,0.0,0.0,4.333333,0.000000,...,0.0,0.0,0.0,NaN,NaN,NaN,0,NaN,0.0,NaN
2,27453088686948352,201911,No,2.745309e+16,2019-11-16 03:09:10,8.0,0.0,0.0,8.000000,0.000000,...,0.0,0.0,0.0,NaN,NaN,NaN,0,NaN,0.0,NaN
3,30294704807493632,201911,E02,NaN,NaT,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,0,NaN,0.0,NaN
4,43370097026809856,201911,No,4.337010e+16,2019-11-16 01:38:50,4.0,0.0,0.0,3.666667,0.000000,...,0.0,0.0,0.0,NaN,NaN,NaN,0,NaN,0.0,NaN


# xh_xw_v3 打分

In [46]:
with open('./model_v3/predictors_use_v3.pt', 'rb') as f:
    predictors_use_v3 = json.load(f)

In [47]:
len(predictors_use_v3)

265

In [48]:
lgb_clf_v3 = joblib.load('./model_v3/train_model_v3.m')

In [49]:
from model_v3 import risk_lvl_v3_100 as rl_v3

In [50]:
dafen_data_v3 = pd.DataFrame()

In [51]:
dafen_data_v3['uid'] = dafen_data.uid.values

In [52]:
dafen_data_v3['prob_score'] = round_6(lgb_clf_v3.predict_proba(dafen_data[predictors_use_v3])[:,1])

In [53]:
dafen_data_v3['risk_level'] = dafen_data_v3['prob_score'].apply(rl_v3.risk_lvl)

In [54]:
dafen_data_v3['score'] = dafen_data_v3['prob_score'].apply(get_score)

In [55]:
dafen_data_v3 = pd.merge(dafen_data_v3,uid_have_score,on='uid',how='left')

In [56]:
v3_final_all = pd.concat([dafen_data_v3,uid_no_score])

In [57]:
v3_final_all.rename(columns={"p3": "statement_period"},inplace=True)

In [58]:
v3_final_all['version'] = 'xh_xw_v3'

In [59]:
v3_final_all

,excl_code,statement_period,prob_score,risk_level,score,uid,version
0,No,201911,0.045316,23,510.0,26880658926223360,xh_xw_v3
1,No,201911,0.006829,87,566.0,26885252134486017,xh_xw_v3
2,No,201911,0.006905,86,565.0,27453088686948352,xh_xw_v3
3,E02,201911,0.092951,7,488.0,30294704807493632,xh_xw_v3
4,No,201911,0.009698,78,555.0,43370097026809856,xh_xw_v3
5,No,201911,0.059707,16,501.0,44677524278493184,xh_xw_v3
6,No,201911,0.013370,68,546.0,46188929970094080,xh_xw_v3
7,No,201911,0.024374,45,528.0,62048738016509952,xh_xw_v3
8,No,201911,0.056151,17,503.0,65548332405899265,xh_xw_v3
9,No,201911,0.039964,27,514.0,69362080106692609,xh_xw_v3


# xh_xw_v3_ntd 打分

In [60]:
with open('./model_v3_ntd/predictors_use_v3_ntd.pt', 'rb') as f:
    predictors_use_v3_ntd = json.load(f)

In [61]:
len(predictors_use_v3_ntd)

256

In [62]:
lgb_clf_v3_ntd = joblib.load('./model_v3_ntd/train_model_v3_ntd.m')

In [63]:
from model_v3_ntd import risk_lvl_v3_ntd_100 as rl_v3_ntd

In [64]:
dafen_data_v3_ntd = pd.DataFrame()

In [65]:
dafen_data_v3_ntd['uid'] = dafen_data.uid.values

In [66]:
dafen_data_v3_ntd['prob_score'] = round_6(lgb_clf_v3_ntd.predict_proba(dafen_data[predictors_use_v3_ntd])[:,1])

In [67]:
dafen_data_v3_ntd['risk_level'] = dafen_data_v3_ntd['prob_score'].apply(rl_v3_ntd.risk_lvl)

In [68]:
dafen_data_v3_ntd['score'] = dafen_data_v3_ntd['prob_score'].apply(get_score)

In [69]:
dafen_data_v3_ntd = pd.merge(dafen_data_v3_ntd,uid_have_score,on='uid',how='left')

In [70]:
v3_ntd_final_all = pd.concat([dafen_data_v3_ntd,uid_no_score])

In [71]:
v3_ntd_final_all.rename(columns={"p3": "statement_period"},inplace=True)

In [72]:
v3_ntd_final_all['version'] = 'xh_xw_v3_ntd'

In [73]:
v3_ntd_final_all

,excl_code,statement_period,prob_score,risk_level,score,uid,version
0,No,201911,0.036535,32,516.0,26880658926223360,xh_xw_v3_ntd
1,No,201911,0.008681,82,559.0,26885252134486017,xh_xw_v3_ntd
2,No,201911,0.006285,89,568.0,27453088686948352,xh_xw_v3_ntd
3,E02,201911,0.094268,6,487.0,30294704807493632,xh_xw_v3_ntd
4,No,201911,0.009113,81,557.0,43370097026809856,xh_xw_v3_ntd
5,No,201911,0.082023,8,492.0,44677524278493184,xh_xw_v3_ntd
6,No,201911,0.014041,68,545.0,46188929970094080,xh_xw_v3_ntd
7,No,201911,0.025393,47,527.0,62048738016509952,xh_xw_v3_ntd
8,No,201911,0.059756,16,501.0,65548332405899265,xh_xw_v3_ntd
9,No,201911,0.061154,15,501.0,69362080106692609,xh_xw_v3_ntd
